# Project Title: Time Series Analysis of Pennsylvania and Illinois Weather, Energy Consumption, and Flu Contagion (2013–2015)

## Introduction

In this project, we aim to identify and explore temporal patterns, correlations, and potential causal relationships between weather conditions, energy consumption, and flu contagion across Pennsylvania and Illinois from 2013 to 2015. By analyzing these interrelated datasets, we hope to derive insights that can inform stakeholders in public health and energy management.

### Objectives
- Analyze daily/hourly weather data to identify trends and seasonal patterns.
- Examine hourly energy consumption data to understand usage patterns in relation to weather.
- Investigate weekly flu contagion data to determine correlations with weather and energy consumption.
- Utilize statistical modeling techniques to assess interdependencies among the datasets.

### Key Datasets
1. **USA Weather Dataset (2013–2015)**:
   - Temporal granularity: Hourly
   - Variables of interest: Temperature.

2. **PJM Historic Energy Consumption**:
   - Temporal granularity: Hourly
   - Scope: Energy usage data for Pennsylvania (Duquesne Light) and Illinois (ComEd).

3. **Flu Contagion Dataset by State (2013–2015)**:
   - Temporal granularity: Weekly
   - Variables: Influenza-like illness (ILI) [activity](https://www.cdc.gov/mmwr/volumes/67/wr/mm6722a4.htm) in the USA.

This notebook will guide you through the data ingestion, preparation, exploratory data analysis (EDA), time series modeling, and visualization phases of the project.

In [1]:
# Data Manipulation and Analysis
import pandas as pd  # For data manipulation and analysis using DataFrames
import numpy as np   # For numerical operations and handling arrays

# Statistical Analysis
import scipy.stats as stats  # For statistical tests and distributions
from statsmodels.tsa.stattools import adfuller  # For stationarity tests

# Time Series Analysis
from statsmodels.tsa.arima.model import ARIMA  # For ARIMA modeling
from statsmodels.tsa.seasonal import seasonal_decompose  # For seasonal decomposition of time series

# Machine Learning Libraries
from sklearn.model_selection import train_test_split  # For splitting datasets into training and testing sets
from sklearn.ensemble import RandomForestRegressor  # For regression tasks
import xgboost as xgb  # For gradient boosting models

# Visualization Libraries
import matplotlib.pyplot as plt  # For creating static visualizations
import seaborn as sns            # For enhanced statistical visualizations
import plotly.express as px      # For interactive plots (optional)

# Date and Time Handling
from datetime import datetime     # For date/time manipulation


## Data Ingestion/Wrangling

We begin by retrieving the relevant dataframes for our job.

Weather dataset, granularity in hours. We are only interested in two states. This dataset is the one imposing the time dataframe in our study. As we are interested in its influence on energy consumption and flu activity, we isolate only the temperature (originally in Kelvin degrees).

Note: for this particular dataset we only have Pittsburgh and Chicago as representative cities for the states of Pennsylvania and Illinois respectivelly.

In [2]:
path = r'sources/historical_hourly_weather_data- 2012_to_2017/temperature.csv'
df = pd.read_csv(path)
display(df)

,datetime,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
0,2012-10-01 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,309.100000,NaN,NaN,NaN
1,2012-10-01 13:00:00,284.630000,282.080000,289.480000,281.800000,291.870000,291.530000,293.410000,296.600000,285.120000,...,285.630000,288.220000,285.830000,287.170000,307.590000,305.470000,310.580000,304.4,304.4,303.5
2,2012-10-01 14:00:00,284.629041,282.083252,289.474993,281.797217,291.868186,291.533501,293.403141,296.608509,285.154558,...,285.663208,288.247676,285.834650,287.186092,307.590000,304.310000,310.495769,304.4,304.4,303.5
3,2012-10-01 15:00:00,284.626998,282.091866,289.460618,281.789833,291.862844,291.543355,293.392177,296.631487,285.233952,...,285.756824,288.326940,285.847790,287.231672,307.391513,304.281841,310.411538,304.4,304.4,303.5
4,2012-10-01 16:00:00,284.624955,282.100481,289.446243,281.782449,291.857503,291.553209,293.381213,296.654466,285.313345,...,285.850440,288.406203,285.860929,287.277251,307.145200,304.238015,310.327308,304.4,304.4,303.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45248,2017-11-29 20:00:00,NaN,282.000000,NaN,280.820000,293.550000,292.150000,289.540000,294.710000,285.720000,...,290.240000,NaN,275.130000,288.080000,NaN,NaN,NaN,NaN,NaN,NaN
45249,2017-11-29 21:00:00,NaN,282.890000,NaN,281.650000,295.680000,292.740000,290.610000,295.590000,286.450000,...,289.240000,NaN,274.130000,286.020000,NaN,NaN,NaN,NaN,NaN,NaN
45250,2017-11-29 22:00:00,NaN,283.390000,NaN,282.750000,295.960000,292.580000,291.340000,296.250000,286.440000,...,286.780000,NaN,273.480000,283.940000,NaN,NaN,NaN,NaN,NaN,NaN
45251,2017-11-29 23:00:00,NaN,283.020000,NaN,282.960000,295.650000,292.610000,292.150000,297.150000,286.140000,...,284.570000,NaN,272.480000,282.170000,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df_temperature = df[['datetime', 'Pittsburgh', 'Chicago']].copy()

# export it as CSV
df_temperature.to_csv('raw_temperature.csv', index=False)

# change col names
df_temperature.columns = ['datetime', 'pittsburgh_temp', 'chicago_temp']

# change 'datetime' format
df_temperature['datetime'] = pd.to_datetime(df_temperature['datetime'])

display(df_temperature)

,datetime,pittsburgh_temp,chicago_temp
0,2012-10-01 12:00:00,NaN,NaN
1,2012-10-01 13:00:00,281.000000,284.010000
2,2012-10-01 14:00:00,281.024767,284.054691
3,2012-10-01 15:00:00,281.088319,284.177412
4,2012-10-01 16:00:00,281.151870,284.300133
...,...,...,...
45248,2017-11-29 20:00:00,285.300000,281.340000
45249,2017-11-29 21:00:00,285.330000,281.690000
45250,2017-11-29 22:00:00,282.910000,281.070000
45251,2017-11-29 23:00:00,280.140000,280.060000


In [4]:
# for completitude we count NaNs and duplicates
# the few NaNs will be absorbed later on the average
print(f'## df_temperature NaN values: \n{df_temperature.isna().sum()}\n')
print(f'## df_temperature dates duplicated values: {df_temperature['datetime'].duplicated().sum()}')

## df_temperature NaN values: 
datetime           0
pittsburgh_temp    3
chicago_temp       3
dtype: int64

## df_temperature dates duplicated values: 0


In [5]:
# check unique values in temperature dataframe
unique_years_temperature = df_temperature['datetime'].dt.year.unique()
print("## Unique years in temperature dataframe:", unique_years_temperature)

unique_months_temperature = df_temperature['datetime'].dt.month.unique()
print("## Unique months in temperature dataframe:", unique_months_temperature)

unique_days_temperature = df_temperature['datetime'].dt.day.unique()
print("## Unique days in temperature dataframe:", unique_days_temperature)

unique_hour_temperature = df_temperature['datetime'].dt.hour.unique()
print("## Unique hour in temperature dataframe:", unique_hour_temperature)

## Unique years in temperature dataframe: [2012 2013 2014 2015 2016 2017]
## Unique months in temperature dataframe: [10 11 12  1  2  3  4  5  6  7  8  9]
## Unique days in temperature dataframe: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31]
## Unique hour in temperature dataframe: [12 13 14 15 16 17 18 19 20 21 22 23  0  1  2  3  4  5  6  7  8  9 10 11]


For the electrical energy consumption by local statal companies, we have a dataset for Duquesne Light Co. (DUQ) and Commonwealth Edison (ComEd). The problem (for our analysis) is the interconnected energy distribution network: we have regional (not city) energy load:

**Duquesne Light Company** primarily provides electricity to southwestern Pennsylvania, specifically serving the greater Pittsburgh area and parts of Allegheny and Beaver counties. The utility serves approximately 600,000 customers across an 817 square mile service area, with about 90% of its service area being residential [[1]](https://www.electricchoice.com/utilities/duquesne-light/), [[2]](https://www.chooseenergy.com/utilities/duquesne-light-co-pa/), [[3]](https://electricityplans.com/pennsylvania/utilities/duquesne-light-company/).

**Commonwealth Edison (ComEd)** is the largest electric utility in Illinois, serving more than 4,000,000 customers across northern Illinois, which represents approximately 70% of the state's population. The company's service territory covers about 11,400 square miles, stretching from the Wisconsin border to the north, Iowa border to the west, Indiana border to the east, and as far south as Iroquois County. ComEd, a unit of Exelon Corporation, manages over 90,000 miles of power lines and has been providing electric service to the region for more than 100 years [[4]](https://www.exeloncorp.com/companies/comed), [[5]](https://www.ilcma.org/friends-of-ilcma/comed/), [[6]](https://en.wikipedia.org/wiki/Commonwealth_Edison).

To bridge the connection between weather and energy datasets, we can hypothesize that state-level temperature trends are closely aligned with the temperature variations observed in their primary metropolitan areas. This assumption allows us to use city temperature data as a reliable proxy for regional weather conditions, facilitating more granular analysis of energy consumption patterns in relation to temperature fluctuations.

In [6]:
# import energy datasets
path1 = r'sources\hourly_energy_consumption\DUQ_hourly.csv'
path2 = r'sources\hourly_energy_consumption\COMED_hourly.csv'
df_pennsylvania = pd.read_csv(path1)
df_illinois = pd.read_csv(path2)

# check df structure
display(df_pennsylvania.head())
print()
display(df_illinois.head())

,Datetime,DUQ_MW
0,2005-12-31 01:00:00,1458.0
1,2005-12-31 02:00:00,1377.0
2,2005-12-31 03:00:00,1351.0
3,2005-12-31 04:00:00,1336.0
4,2005-12-31 05:00:00,1356.0


,Datetime,COMED_MW
0,2011-12-31 01:00:00,9970.0
1,2011-12-31 02:00:00,9428.0
2,2011-12-31 03:00:00,9059.0
3,2011-12-31 04:00:00,8817.0
4,2011-12-31 05:00:00,8743.0


Although [Pennsylvania](https://www.macrotrends.net/global-metrics/states/pennsylvania/population) and [Illinois](https://www.macrotrends.net/global-metrics/states/illinois/population) have similar populations, there is a magnitude order in energy load.

| Year | Pennsylvania | Illinois    |
|------|--------------|-------------|
| 2012 | 12,763,536   | 12,875,280  |
| 2013 | 12,773,801   | 12,882,250  |
| 2014 | 12,787,209   | 12,880,552  |
| 2015 | 12,802,503   | 12,859,585  |
| 2016 | 12,784,227   | 12,821,709  |
| 2017 | 12,805,537   | 12,779,893  |


So to normalize the electric energy load, we will need to divide by the amount of clients of each company:

In [7]:
# manipulate energy datasets
df_pennsylvania_pop = df_pennsylvania.copy()
df_pennsylvania_pop['Duquesne_kW'] = df_pennsylvania_pop['DUQ_MW'] / 600 # divide by customers and convert to kW
df_pennsylvania_pop.drop(columns='DUQ_MW', inplace=True)
display(df_pennsylvania_pop)
print()

df_illinois_pop = df_illinois.copy()
df_illinois_pop['ComEdison_kW'] = df_illinois_pop['COMED_MW'] / 4000 # divide by customers and convert to kW
df_illinois_pop.drop(columns='COMED_MW', inplace=True)
display(df_illinois_pop)

,Datetime,Duquesne_kW
0,2005-12-31 01:00:00,2.430000
1,2005-12-31 02:00:00,2.295000
2,2005-12-31 03:00:00,2.251667
3,2005-12-31 04:00:00,2.226667
4,2005-12-31 05:00:00,2.260000
...,...,...
119063,2018-01-01 20:00:00,3.270000
119064,2018-01-01 21:00:00,3.233333
119065,2018-01-01 22:00:00,3.151667
119066,2018-01-01 23:00:00,3.033333


,Datetime,ComEdison_kW
0,2011-12-31 01:00:00,2.49250
1,2011-12-31 02:00:00,2.35700
2,2011-12-31 03:00:00,2.26475
3,2011-12-31 04:00:00,2.20425
4,2011-12-31 05:00:00,2.18575
...,...,...
66492,2018-01-01 20:00:00,3.46450
66493,2018-01-01 21:00:00,3.43950
66494,2018-01-01 22:00:00,3.40675
66495,2018-01-01 23:00:00,3.33400


There are some duplicated values, much possible due to summer time:

In [8]:
# check missing values
print(f'## Pensylvania missing values: \n{df_pennsylvania_pop.isna().sum()}')
print()
print(f'## Illinois missing values: \n{df_illinois_pop.isna().sum()}')
print()

# check duplicates
print(f'## Pensylvania duplicated values: \n{df_pennsylvania_pop['Datetime'].duplicated().sum()}')
print()
print(f'## Illinois duplicated values: \n{df_illinois_pop['Datetime'].duplicated().sum()}')

## Pensylvania missing values: 
Datetime       0
Duquesne_kW    0
dtype: int64

## Illinois missing values: 
Datetime        0
ComEdison_kW    0
dtype: int64

## Pensylvania duplicated values: 
4

## Illinois duplicated values: 
4


In [9]:
# remove duplicates
df_pennsylvania_pop = df_pennsylvania_pop.drop_duplicates(subset='Datetime')
df_illinois_pop = df_illinois_pop.drop_duplicates(subset='Datetime')

# check duplicates
print(f'## Pensylvania duplicated values: \n{df_pennsylvania_pop['Datetime'].duplicated().sum()}')
print()
print(f'## Illinois duplicated values: \n{df_illinois_pop['Datetime'].duplicated().sum()}')

## Pensylvania duplicated values: 
0

## Illinois duplicated values: 
0


In [10]:
# ensure 'Datetime' is in datetime format
df_pennsylvania_pop['Datetime'] = pd.to_datetime(df_pennsylvania_pop['Datetime'])
df_illinois_pop['Datetime'] = pd.to_datetime(df_illinois_pop['Datetime'])

# check unique years in Pennsylvania dataframe
unique_years_pennsylvania = df_pennsylvania_pop['Datetime'].dt.year.unique()
print("## Unique years in Pennsylvania dataframe:", unique_years_pennsylvania)

# check unique years in Illinois dataframe
unique_years_illinois = df_illinois_pop['Datetime'].dt.year.unique()
print("## Unique years in Illinois dataframe:", unique_years_illinois)

## Unique years in Pennsylvania dataframe: [2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018]
## Unique years in Illinois dataframe: [2011 2012 2013 2014 2015 2016 2017 2018]


We need only the time period specified by the weather data. We achieve that with a left inner join on the temperature dataset:

In [11]:
# merge temp and duqesne
df_energy = pd.merge(df_temperature, df_pennsylvania_pop, left_on='datetime', right_on='Datetime', how='inner')
df_energy.drop(columns='Datetime', inplace=True)

# merge with common edison
df_energy = pd.merge(df_energy, df_illinois_pop, left_on='datetime', right_on='Datetime', how='inner')
df_energy.drop(columns='Datetime', inplace=True)

# check shape
display(df_energy)
display(f'## Shape of df_temperature:{df_temperature.shape}')
display(f'## Shape of df_energy:{df_energy.shape}')

# check missin values
print(f'## df_energy missing values: \n{df_energy.isna().sum()}')

,datetime,pittsburgh_temp,chicago_temp,Duquesne_kW,ComEdison_kW
0,2012-10-01 12:00:00,NaN,NaN,2.716667,2.82550
1,2012-10-01 13:00:00,281.000000,284.010000,2.733333,2.85650
2,2012-10-01 14:00:00,281.024767,284.054691,2.745000,2.88050
3,2012-10-01 15:00:00,281.088319,284.177412,2.716667,2.89350
4,2012-10-01 16:00:00,281.151870,284.300133,2.711667,2.87225
...,...,...,...,...,...
45241,2017-11-29 20:00:00,285.300000,281.340000,2.690000,3.04275
45242,2017-11-29 21:00:00,285.330000,281.690000,2.676667,2.99950
45243,2017-11-29 22:00:00,282.910000,281.070000,2.591667,2.92025
45244,2017-11-29 23:00:00,280.140000,280.060000,2.461667,2.78700


'## Shape of df_temperature:(45253, 3)'

'## Shape of df_energy:(45246, 5)'

## df_energy missing values: 
datetime           0
pittsburgh_temp    3
chicago_temp       3
Duquesne_kW        0
ComEdison_kW       0
dtype: int64


After performing an intersection on the temperature datetime, the number of rows was reduced by 7 due to the use of daylight saving time. And is the same reason why we have 3 missing temperature values. This would imply a 1 hour phase shift on the modeling during daylight saving time.

Flu dataset, pay attention to its granularity in weeks. The relationship between the weather dataset and flu activity dataset is not straight forward and need grouping.

![alt text](<erd diagram.png>)

In [ ]:
path = r'sources\fluview\StateDatabySeason55_54,53,52,57,56.csv'
df = pd.read_csv(path)
display(df)

In [ ]:
df_flu = df[(df['STATENAME'] == 'Pennsylvania') | (df['STATENAME'] == 'Illinois')].copy()
display(df_flu)
df_flu.to_csv('raw_flu.csv', index=False)